# Curate Datasets

### Read the datasets and import the libraries

In [2]:
# Import our libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import seaborn as sns
sns.set(style="ticks")

%matplotlib inline

# Read in our dataset
mastercard = pd.read_csv('MasterCardData\GeoInsights_Synthetic_Output.csv')

# Take a look at the first few rows of the dataset
mastercard.head()

,txn_date,industry,quad_id,txn_amt,txn_cnt,acct_cnt,avg_ticket,avg_freq,avg_spend_amt,yoy_txn_amt,yoy_txn_cnt
0,2022-01-01,Total Retail,120230021323230,163.34,335.39,381.79,26.04,99.97,40.81,97.32,37.49
1,2022-01-01,Total Retail,120230023131110,400.82,474.70,462.13,120.85,104.33,106.34,-50.11,-75.64
2,2022-01-01,Eating Places,120230023131110,263.75,241.53,240.95,82.35,120.39,133.75,-31.29,-31.88
3,2022-01-01,Total Apparel,120230023131110,162.59,229.95,252.82,127.35,98.36,111.11,-16.39,-51.05
4,2022-01-01,Total Retail,120230210333222,171.27,128.84,131.45,425.06,100.94,429.76,212.50,-41.44


In [14]:
# Read in our dataset
geo_data = pd.read_csv('MasterCardData\GeoInsights_Hackathon_Quads_GeoInfo.csv', delimiter='|')
geo_data.head()

,quad_id,central_latitude,central_longitude,bounding_box
0,120230013313003,48.155093,13.991089,"POLYGON ((13.99658203125 48.15142814322122, 13..."
1,120230021330223,47.580231,12.496948,"POLYGON ((12.50244140625 47.57652571374621, 12..."
2,120230023133222,47.282955,12.573853,"POLYGON ((12.579345703125 47.27922900257082, 1..."
3,120230123333302,47.058896,15.430298,"POLYGON ((15.435791015625 47.05515408550347, 1..."
4,120221133132132,47.312759,11.145630,"POLYGON ((11.151123046875 47.30903424774781, 1..."


### Get the quad_ids of the zones within a radius of the event locations

In [15]:
# Haversine formula to calculate distance between two lat-lon points
def haversine(lon1, lat1, lon2, lat2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert degrees to radians
    lon1_radians = np.radians(lon1)
    lat1_radians = np.radians(lat1)
    lon2_radians = np.radians(lon2)
    lat2_radians = np.radians(lat2)
    
    # Difference in coordinates
    dlon = lon2_radians - lon1_radians
    dlat = lat2_radians - lat1_radians
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_radians) * np.cos(lat2_radians) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Function to filter the dataset
def filter_by_radius(dataframe, event_location, radius):
    """
    dataframe: pandas DataFrame containing 'latitude' and 'longitude' columns
    event_location: tuple of (latitude, longitude) for the event location
    radius: radius in kilometers
    """
    # Apply the haversine function to each row in the dataframe
    distances = dataframe.apply(lambda row: haversine(event_location[1], event_location[0],
                                                      row['central_longitude'], row['central_latitude']), axis=1)
    
    # Filter the dataframe for rows where the distance is less than or equal to the radius
    return dataframe[distances <= radius]

In [25]:
# Calculate the middle point of Point A and Point B
def get_middlepoint(point_a, point_b):
    # Calculate the midpoint
    midpoint_longitude = (point_a[0] + point_b[0]) / 2
    midpoint_latitude = (point_a[1] + point_b[1]) / 2
    midpoint = (midpoint_latitude, midpoint_longitude)
    return midpoint

In [37]:
# Event coordinates
adidas_location = (47.169753170157186, 13.106657266616821)
snowboard_location = get_middlepoint((47.110209967822342, 13.132610321044922), (47.106588081374838, 13.122954368591309))
red_bull_location = get_middlepoint((47.112542887152543, 13.133828043937683), (47.115237117435001, 13.133377432823181))
ski_location = (47.062163427549223, 13.058323860168455)

In [60]:
# Filtered data
R = 10 # Radius in kilometers
ski_filtered_df = filter_by_radius(geo_data, ski_location, R)
snowboard_filtered_df = filter_by_radius(geo_data, snowboard_location, R)
red_bull_filtered_df = filter_by_radius(geo_data, red_bull_location, R)
adidas_filtered_df = filter_by_radius(geo_data, adidas_location, R)

In [64]:
red_bull_filtered_df

,quad_id,central_latitude,central_longitude,bounding_box
18,120230032321003,47.148633,13.112183,"POLYGON ((13.11767578125 47.14489748555397, 13..."
92,120230032321213,47.118738,13.134155,"POLYGON ((13.1396484375 47.11499982620772, 13...."
550,120230032321212,47.118738,13.123169,"POLYGON ((13.128662109375 47.11499982620772, 1..."
1010,120230032323012,47.088825,13.123169,"POLYGON ((13.128662109375 47.08508535995384, 1..."
1028,120230032303220,47.171044,13.101196,"POLYGON ((13.106689453125 47.16730970131577, 1..."
1064,120230032321231,47.111261,13.134155,"POLYGON ((13.1396484375 47.10752278534249, 13...."
1143,120230032303202,47.178512,13.101196,"POLYGON ((13.106689453125 47.17477833929903, 1..."
1998,120230032321000,47.156105,13.101196,"POLYGON ((13.106689453125 47.15236927446392, 1..."
2019,120230032303223,47.163575,13.112183,"POLYGON ((13.11767578125 47.15984001304431, 13..."
2059,120230032302331,47.171044,13.090210,"POLYGON ((13.095703125 47.16730970131577, 13.0..."


### Filter the Mastercard data by time of the event

In [ ]:
red_bull_mastercard = mastercard[mastercard['quad_id'].isin(red_bull_filtered_df['quad_id'])]
red_bull_mastercard

In [61]:
ski_mastercard_time_filtered = mastercard[(mastercard['txn_date'] >= '2022-12-10') & (mastercard['txn_date'] <= '2022-12-11')]
snowboard_22_mastercard_time_filtered = mastercard[(mastercard['txn_date'] >= '2022-01-11') & (mastercard['txn_date'] <= '2022-01-12')]
snowboard_23_mastercard_time_filtered = mastercard[(mastercard['txn_date'] >= '2023-01-10') & (mastercard['txn_date'] <= '2023-01-11')]
adidas_mastercard_time_filtered = mastercard[(mastercard['txn_date'] == '2022-09-10')]
red_bull_mastercard_time_filtered = mastercard[(mastercard['txn_date'] == '2023-02-10')]

In [62]:
# Perform an inner join operation on the 'quad_id' column
ski_mastercard = ski_mastercard_time_filtered[ski_mastercard_time_filtered['quad_id'].isin(ski_filtered_df['quad_id'])]
snowboard_22_mastercard = snowboard_22_mastercard_time_filtered[snowboard_22_mastercard_time_filtered['quad_id'].isin(snowboard_filtered_df['quad_id'])]
snowboard_23_mastercard = snowboard_23_mastercard_time_filtered[snowboard_23_mastercard_time_filtered['quad_id'].isin(snowboard_filtered_df['quad_id'])]
adidas_mastercard = adidas_mastercard_time_filtered[adidas_mastercard_time_filtered['quad_id'].isin(adidas_filtered_df['quad_id'])]
red_bull_mastercard = red_bull_mastercard_time_filtered[red_bull_mastercard_time_filtered['quad_id'].isin(red_bull_filtered_df['quad_id'])]


In [63]:
# ski_mastercard.head()
# snowboard_22_mastercard.head()
# snowboard_23_mastercard.head()
# adidas_mastercard.head()
red_bull_mastercard.head()

,txn_date,industry,quad_id,txn_amt,txn_cnt,acct_cnt,avg_ticket,avg_freq,avg_spend_amt,yoy_txn_amt,yoy_txn_cnt
80672,2023-02-10,Total Retail,120230032303223,76.70,74.34,78.51,233.47,96.97,254.92,227.33,61.97
256223,2023-02-10,Eating Places,120230032303220,135.40,143.79,141.72,107.37,99.46,110.72,69.19,145.41
256224,2023-02-10,Accommodations,120230032303220,123.69,110.33,65.26,179.45,100.93,150.83,0.52,113.95
256226,2023-02-10,Total Apparel,120230032303220,101.83,65.67,69.98,72.72,99.06,78.79,114.26,105.25
256227,2023-02-10,Total Retail,120230032303220,305.00,207.25,204.90,168.99,100.95,162.50,51.06,85.12
